# Фундаментальный анализ акций

In [1]:
# Библиотеки
import pandas as pd
import numpy as np

import requests
import urllib.request
from urllib.parse import urlencode
from urllib.request import urlopen

from bs4 import BeautifulSoup as bs
import json
import re
import io
import os
import time

import datetime as dt

import datetime
from datetime import datetime, date, timedelta

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
# загружаем ставку ЦБ на сегодня
url = 'https://cbr.ru/hd_base/KeyRate/'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]
cb = df.iloc[0][1]/100
print(f'Ключевая ставка = {cb} %')

Ключевая ставка = 15.0 %


In [4]:
def Q_date(date):
    Q_list = {'4': '12-31', '3': '09-30', '2': '06-30', '1': '03-31'}
    if date == 'LTM':
        return date
    else:
        date_list = date.split('Q')
        Q = Q_list[date_list[1]]
        return date_list[0] +'-' + Q

In [5]:
# загружаем квартальные финансовые отчеты МСФО
def MSFO_stock(stock = "SBER"):
    url =f'https://smart-lab.ru/q/{stock}/f/q/MSFO/download/'
    req = urllib.request.Request(
    url,
    data=None,
    headers={
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    }
    )
    s = urllib.request.urlopen(req)
    df = pd.read_csv(s , sep=';', header=0)
    
    df = df.T
    df.columns = df.iloc[0]
    df = df.iloc[1:].rename_axis(None, axis=1)
    
    if stock == "SBER":
        df = df[['Капитализация, млрд руб', 'EV, млрд руб', 'Чистый операц доход, млрд руб', 'Чистая прибыль, млрд руб', 'Див доход, ао, %', 'Дивиденды/прибыль, %', 'P/E', 'P/B', 'ROE, %', 'ROA, %', 'Чистая процентная маржа, %']]
    else:
        df = df[['Капитализация, млрд руб', 'EV, млрд руб', 'Выручка, млрд руб', 'Чистая прибыль, млрд руб', 'Див доход, ао, %', 'Дивиденды/прибыль, %', 'P/E', 'P/S', 'P/BV', 'ROE, %', 'ROA, %', 'EV/EBITDA', 'Рентаб EBITDA, %', 'Долг/EBITDA']]
    
    df = df.reset_index()
    df['index'] = df['index'].apply(lambda x: Q_date(x))
    df = df.set_index('index')
    df.index.names = ['DATE']
    df = df.loc['2013-12-31':]

    return df

In [6]:
stock = ['GAZP', 'ROSN', 'SBER', 'RTKM', 'MOEX']

In [7]:
for i in stock:
    MSFO_stock(stock = i).to_csv(f'MSFO_{i}.csv', index = True)